In [1]:
from embedding.vit4embedding import ViTForEmbedding,VitConfigForEmbedding
from transformers import AutoImageProcessor
from PIL import Image
import numpy as np 
import torch as t 

In [2]:
model_name_or_path = 'models/google/vit-large-patch16-224-in21k'


config = VitConfigForEmbedding.from_pretrained(model_name_or_path,embedding_dim=1024)

image_embedding_model = ViTForEmbedding.from_pretrained(pretrained_model_name_or_path=model_name_or_path,config=config, device_map='cuda:0')
image_processor = AutoImageProcessor.from_pretrained(model_name_or_path)



Some weights of ViTForEmbedding were not initialized from the model checkpoint at models/google/vit-large-patch16-224-in21k and are newly initialized: ['embeddings.mask_token', 'output_linear.bias', 'output_linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [3]:
images_list = [Image.open(i).convert('RGB') for i in [
    'data/test_data/截屏2024-11-21 13.20.05.png'
]]
encoded_input = image_processor(images=images_list, return_tensors="pt")

for k in encoded_input.keys():
    encoded_input[k] = encoded_input[k].to(image_embedding_model.device)

last_hidden_states, pool_output = image_embedding_model(**encoded_input)

In [4]:
pool_output.shape

torch.Size([1, 1024])

In [5]:
pool_output

tensor([[ 0.0101,  0.2668, -0.0197,  ...,  0.1814,  0.1021,  0.0779]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [6]:
image_embedding_model.save_pretrained("models/test/image_embedding_001")
image_processor.save_pretrained("models/test/image_embedding_001")

[2024-12-03 07:04:38,754] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


['models/test/image_embedding_001/preprocessor_config.json']

In [7]:
## 保存一下模型，用于后面训练
final_image_embedding = "models/image_embedding/image_embedding_001"
image_embedding_model.save_pretrained(final_image_embedding)
image_processor.save_pretrained(final_image_embedding)

['models/image_embedding/image_embedding_001/preprocessor_config.json']

In [8]:
image_embedding_model2 = ViTForEmbedding.from_pretrained(pretrained_model_name_or_path='models/test/image_embedding_001', device_map='cuda:0')


In [9]:

last_hidden_states, pool_output2 = image_embedding_model2(**encoded_input)
pool_output2.shape


torch.Size([1, 1024])

In [10]:
pool_output2

tensor([[ 0.0101,  0.2668, -0.0197,  ...,  0.1814,  0.1021,  0.0779]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [11]:
t.allclose(pool_output,pool_output2)

True

In [15]:
last_hidden_states + pool_output2[:, : image_embedding_model2.config.hidden_size] 

tensor([[-0.6441,  0.0309,  0.8128,  ..., -0.0981, -0.1634, -0.0594]],
       device='cuda:0', grad_fn=<AddBackward0>)

tensor([[-0.6542, -0.2359,  0.8325,  ..., -0.2795, -0.2655, -0.1374]],
       device='cuda:0', grad_fn=<SelectBackward0>)